In [2]:
from openpyxl import load_workbook
import pandas as pd
import re

In [3]:
file_input = input('introduce the path file and name: ').strip('"')
def premiums(sheets,cells):
    file = load_workbook(file_input,data_only=False)
    sheet = file[sheets]
    formula = sheet[cells].value

    formula_text = (formula or "").lstrip("=")
    formula_text = formula_text.split('+',1)[1].strip() if '+' in formula_text else formula_text

    token = r'(?:\$?[A-Z]{1,3}\$?\d+|[+-]?\d*\.?\d+)'  # cell or number

    pattern = rf'([+-]?\d*\.?\d+)\s*\*\s*\(\s*({token})\s*/\s*({token})'
    findings = re.findall(pattern,formula_text)

    df = pd.DataFrame(findings,columns=['premium','factor_volume','total_volume'])
    df['premium'] = pd.to_numeric(df['premium'],errors="raise")

    file_2 = load_workbook(file_input,data_only=True)
    sheet_2 = file_2[sheets]
  
    # Search for letters (Capital letters followed by digits, ej: C114)
    cell_ref_pattern = re.compile(r'^\$?[A-Z]{1,3}\$?\d+$')
    
    def resolve_token(x):
        x = str(x).strip()
        if cell_ref_pattern.match(x):
            x = x.replace("$", "")
            return sheet_2[x].value
        return float(x)

    df["factor_volume"] = df["factor_volume"].apply(resolve_token)
    df['total_volume'] = df['total_volume'].apply(resolve_token)
    return df

In [4]:
sheet_cells_dict = {
    'NA SBO 2025':{'G67':{'january':'q1'},'G114':{'february':'q1'},'G160':{'march':'q1'},'G212':{'april':'q2'},'G264':{'may':'q2'},'G318':{'june':'q2'},
                   'G373':{'july':'q3'},'G431':{'august':'q3'},'G485':{'september':'q3'},'G540':{'october':'q4'},'G586':{'november':'q4'},'G641':{'december':'q4'}},
    'NA SBO 2026':{'G67':{'january':'q1'},'G114':{'february':'q1'},'G160':{'march':'q1'},'G212':{'april':'q2'},'G264':{'may':'q2'},'G318':{'june':'q2'},
                   'G373':{'july':'q3'},'G431':{'august':'q3'},'G485':{'september':'q3'},'G540':{'october':'q4'},'G586':{'november':'q4'},'G641':{'december':'q4'}},
}

all_premiums = []  # List to store all the dataframes

for sheet_name, cells_dict in sheet_cells_dict.items():
    for cell, periods in cells_dict.items():  # Iterate over HEACH cell from the list
        month, qtrs = list(periods.items())[0]
        try:
            df_result = premiums(sheet_name, cell)
            # Optional: add a column to know from which sheet and cell the value came.
            if not df_result.empty:
                df_result['source_sheet'] = sheet_name
                df_result['month'] = month
                df_result['qtr'] = qtrs
                all_premiums.append(df_result)
        except Exception as e:
            print(f"Error en {sheet_name} celda {cell}: {e}")

# Join in a final DataFrame all the tables stored by the initial variable all_premiums.
if all_premiums:
    final_df = pd.concat(all_premiums, ignore_index=True)
        
else:
    print("No se encontraron datos.")

In [5]:
# Calculate the proportional of premium based in % of volume
final_df['premium_allocation'] = final_df['premium']*(final_df['factor_volume']/final_df['total_volume'])

# moving identifier columns to the beggining of the df.
col_move = ['source_sheet','month','qtr']
new_order = col_move + [col for col in final_df.columns if col not in col_move]
final_df = final_df[new_order]

# build the summary by quarters.
final_df_grouped = final_df.groupby(['source_sheet','qtr'])['premium_allocation'].sum().sort_index(ascending=True)
final_df_grouped = final_df_grouped.unstack('qtr').reset_index()

In [6]:
final_df_grouped

qtr,source_sheet,q1,q2,q3
0,NA SBO 2025,1.59972,0.631408,0.074889
